In [ ]:
import os, io
import pytz
import urllib, json
import datetime as dt
import pandas as pd
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select, and_
from settings.db import WAREHOSUE_URL
from settings.apis import FORECAST_API_SETTINGS

engine_warehouse = create_engine(WAREHOSUE_URL, poolclass=NullPool)
api_base_url = f"{FORECAST_API_SETTINGS['host']}:{FORECAST_API_SETTINGS['port']}/api/forecast/"
api_headers = {FORECAST_API_SETTINGS['api_key_name']: FORECAST_API_SETTINGS['api_key_value']}

In [ ]:
#Select here site to get forecasts for
from settings.sites import kherson as sites_list

query = f'''SELECT site_name, w_code FROM dim_site WHERE site_name in ({str(sites_list)[1:-1]})'''
print(query)
with engine_warehouse.connect() as connection:
    sites = dict(connection.execute(query).fetchall())

In [ ]:
#Set here forecast type and dates
forecast_type = 'applied/final'

dates_list = pd.date_range(start='2022-05-30', end='2022-05-31').to_list()


In [ ]:
target_folder = 'data/forecasts/{}_{}/'.format(dates_list[0].strftime('%Y-%m-%d'), dates_list[-1].strftime('%Y-%m-%d'))
print(target_folder)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

In [ ]:
forecasts = dict()

for site, w_code in sites.items():
    forecasts.update({site: list()})
    for date in dates_list:
        date_str = date.strftime('%Y-%m-%d')
        api_url = api_base_url + f"{forecast_type}/{w_code}/{date_str}"
        request = urllib.request.Request(api_url, headers=api_headers, method='GET')
        with urllib.request.urlopen(request) as response:
            data = json.loads(response.read().decode('utf-8'))['final_forecast']
            forecast = pd.DataFrame(index=range(1, len(data['forecast_values']) + 1), data=data['forecast_values'], columns=[date_str])
            forecasts[site].append(forecast)
    forecasts.update({site: pd.concat(forecasts[site], axis=1)})
    with pd.ExcelWriter(target_folder + f'{site}_forecast.xlsx', engine="openpyxl") as  writer:
        forecasts[site].to_excel(writer, 'results_daily')